# Conversión del archivo de predicción

In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from io import StringIO
import os
import re
import matplotlib.pyplot as plt
import pickle

from time import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_test_file = '020_Text_predict_inicial.csv'
df_nuevo = pd.read_csv(df_test_file)
print(df_nuevo.shape)
df_nuevo.head()

(536165, 6)


,tarea,clase_original,texto_original,texto_etl,texto_corregido,id_clase
0,5408419731,tecnico,Se cambio medidor por error mayor al admisible,cambio medidor error mayor admisible,se cambio medidor por error mayor al admisible,1
1,3904391033,sin anomalias,Med sin anomalias,medidor sin anomalia,medidor sin anomalias,6
2,2773273015,sin anomalias,Galpon cerrado en venta,galpon cerrado venta,galpon cerrado en venta,6
3,2538639960,fraude,se cortó conexión directa . no se normaliza cl...,corto conexion directo no normalizo cliente si...,se corto conexion directa no se normaliza cli...,0
4,3940220116,tecnico,Se cambio medidor quemado,cambio medidor quemado,se cambio medidor quemado,1


In [3]:
#Eliminar columna tarea
df_nuevo.drop('tarea', inplace=True, axis=1)
#cambiar identificadores
abr = {'fraude': 'intrusion',
       'tecnico': 'reparacion',
       'comercial': 'marketing',
       'ya realizada': 'visitada',
       'no concretada': 'sin_realizar',
       'seguridad en la via publica':'seguridad',
       'sin anomalias':'ok_equipo',
       'sin determinar':'sin_tarea'
}
df_nuevo['clase_original'] = df_nuevo['clase_original'].map(abr)
df_nuevo.head(20)

,clase_original,texto_original,texto_etl,texto_corregido,id_clase
0,reparacion,Se cambio medidor por error mayor al admisible,cambio medidor error mayor admisible,se cambio medidor por error mayor al admisible,1
1,ok_equipo,Med sin anomalias,medidor sin anomalia,medidor sin anomalias,6
2,ok_equipo,Galpon cerrado en venta,galpon cerrado venta,galpon cerrado en venta,6
3,intrusion,se cortó conexión directa . no se normaliza cl...,corto conexion directo no normalizo cliente si...,se corto conexion directa no se normaliza cli...,0
4,reparacion,Se cambio medidor quemado,cambio medidor quemado,se cambio medidor quemado,1
5,reparacion,Tapa rota,tapa rota,tapa rota,1
6,seguridad,Se realizó inspección y se cambió caja monopos...,realizo inspeccion cambio caja monoposte rota ...,se realizo inspeccion y se cambio caja monopo...,5
7,reparacion,Se cambio med monof quemado,cambio medidor monofasico quemado,se cambio medidor monofasico quemado,1
8,reparacion,Se cambio medidot mal funsionsmiento de arranke,cambio medidor mal funcionamiento arranque,se cambio medidor mal funcionamiento de arranque,1
9,ok_equipo,Cliente ya normalizado con medidor n 3686465 e...,cliente ya normalizado con medidor estado,cliente ya normalizado con medidor n estado y,6


In [4]:
#Grabar nuevo archivo

#df_nuevo.to_csv('020_Text_predict_final.csv', index=False)

In [14]:
df_test_file = '020_Text_predict_final.csv'
df_nuevo = pd.read_csv(df_test_file)
#### Sample
df_sample = df_nuevo.sample(n = 1000,random_state=0)
df_sample.to_csv('020_Text_predict_final_sample_1000_20220531.csv', index=False)
print(df_sample.shape)
df_sample.head()

(1000, 5)


,clase_original,texto_original,texto_etl,texto_corregido,id_clase
245933,ok_equipo,Se encontro normal se cambio tapa rectangular ...,encontro normal cambio tapa rectangular chica,se encontro normal se cambio tapa rectangular...,6
217279,ok_equipo,A garita le pasan suministro desde vivienda. E...,garita pasan suministro desde vivienda emplead...,a garita le pasan suministro desde vivienda e...,6
56774,seguridad,Se verifico instalacion y carga del medidor si...,verifico instalacion carga medidor sin anomali...,se verifico instalacion y carga del medidor s...,5
170570,reparacion,Se cargó por segunda vez a womSe cambio medido...,cargo segunda vez womse cambio medidor sistema...,se cargo por segunda vez a womse cambio medid...,1
131622,sin_tarea,Nombre:retamar\nMovil:470\nLegajo ayudante:14091,sin determinar,nombre retamar movil legajo ayudante,7


In [9]:
## Se rescata
import tensorflow as tf
from tensorflow.keras.models import save_model, load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
#modelo = '030_textos_modelo_lstm_' + str(limite_de_palabras) + '_palabras.h5'
#modelo = '030_textos_modelo_lstm_20_palabras.h5'
modelo = '030_textos_modelo_lstm_todas_las_palabras.h5'
model = tf.keras.models.load_model(modelo, custom_objects=None, compile=False)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [10]:
df_resultado = pd.DataFrame()

nro_registros_a_procesar= 100
i = 0

for index, row in df_nuevo.iterrows():

    clase_original = row['clase_original']
    #grp_anomalia = row['Grupo Original']

    #nuevo_texto = [row['texto_etl']]
    nuevo_texto = [row['texto_corregido']]
    seq = tokenizer.texts_to_sequences(nuevo_texto)
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

    pred = model.predict(padded)
    labels = ['intrusion', 'reparacion', 'marketing', 'visitada', 'sin_realizar', 'seguridad',
          'ok_equipo', 'sin_tarea']    
    maximo = max(pred[0,0],pred[0,1],pred[0,2],pred[0,3],pred[0,4],pred[0,5],pred[0,6],pred[0,7])
    registro = {'Clase.Original':clase_original,
        'Clase.LSTM':labels[np.argmax(pred)],
        'Certeza.Maxima': maximo,
        'pr.intrusion': [pred[0,0]], 'pr.reparacion': [pred[0,1]],
        'pr.marketing':[pred[0,2]], 'pr.visitada': [pred[0,3]],      
        'pr.sin_realizar':[pred[0,4]], 'pr.seguridad': [pred[0,5]],
        'pr.ok_equipo':[pred[0,6]],'pr.sin_tarea':[pred[0,7]],
        'texto':nuevo_texto}

    #df_append = pd.DataFrame(registro)

    #df_resultado = df_resultado.append(df_append, ignore_index=True) 
    df_resultado = df_resultado.append(registro, ignore_index=True)  

    i = i + 1
    if i == nro_registros_a_procesar:
        break

In [11]:
df_resultado.head(100)

,Clase.Original,Clase.LSTM,Certeza.Maxima,pr.intrusion,pr.reparacion,pr.marketing,pr.visitada,pr.sin_realizar,pr.seguridad,pr.ok_equipo,pr.sin_tarea,texto
0,reparacion,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se cambio medidor por error mayor al admisible]
1,ok_equipo,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ medidor sin anomalias]
2,ok_equipo,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ galpon cerrado en venta]
3,intrusion,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se corto conexion directa no se normaliza cl...
4,reparacion,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se cambio medidor quemado]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,ok_equipo,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se verifico local cerrado encontrandose con ...
96,seguridad,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se coloco taparec trifasica]
97,ok_equipo,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ inspec ocular normal]
98,ok_equipo,marketing,0.730979,[0.01746873],[0.02744228],[0.7309788],[0.0882449],[0.019792087],[0.02044655],[0.017657587],[0.077969044],[ se realizo inspeccion sin anomalia medidor f...
